# Install libraries, import data

In [ ]:
#data
!git clone https://github.com/udacity/CarND-LaneLines-P1.git
from distutils.dir_util import copy_tree
import shutil
copy_tree("./CarND-LaneLines-P1/test_images", "./test_images")
copy_tree("./CarND-LaneLines-P1/test_videos", "./test_videos")
shutil.rmtree('./CarND-LaneLines-P1', ignore_errors=False, onerror=None)

Cloning into 'CarND-LaneLines-P1'...
remote: Enumerating objects: 254, done.
remote: Total 254 (delta 0), reused 0 (delta 0), pack-reused 254
Receiving objects: 100% (254/254), 43.44 MiB | 42.01 MiB/s, done.
Resolving deltas: 100% (122/122), done.


In [ ]:
!pip install moviepy

In [ ]:
#import libraries 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2 as cv
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Functions used for lane detection

In [ ]:
# mask the unnecessary portion of the image

def bounded_region_mask(img, vertices):
    #a numpy array of size same as image, filled with zeroes initially
    mask = np.zeros_like(img)  
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape)<=2:
        bounded_region_shape = 255   #for grayscale 
    else:
        no_of_colors = img.shape[2]  # 3 if RGB 
        bounded_region_shape = (255,)*no_of_colors
        
    #fill the polygon 
    cv.fillPoly(mask,vertices, bounded_region_shape)
    
    #returning the image only where mask pixels are nonzero 
    masked_image = cv.bitwise_and(img, mask)
    return masked_image

In [ ]:
# function to draw lines, by default: red lines with 10 thickness

def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv.line(img, (x1, y1), (x2, y2), color, thickness)

In [ ]:
# for slope of the lines (of the polygon which will enclose the lane)

def slope_lines(image,lines):
    img = image.copy()
    pos_slope = [] 
    neg_slope = [] 

    for line in lines:
        for x1,y1,x2,y2 in line:

            if x1 == x2:
                pass #slope = infinity
            else:
                m = (y2 - y1) / (x2 - x1) #slope

                c = y1 - m * x1 # y= m*x + c

                if m < 0:
                    pos_slope.append((m,c))

                elif m >= 0:
                    neg_slope.append((m,c))

    pos_line= np.mean(pos_slope, axis=0)
    neg_line = np.mean(neg_slope, axis=0)

    poly_vertices = []

    for slope, intercept in [pos_line, neg_line]:

        rows= image.shape[0]
        cols= image.shape[1]

        #getting complete height of image in y1  
        y1= int(rows) #image.shape[0]

        #taking y2 upto 60% of actual height or 60% of y1
        y2= int(rows*0.6) #int(0.6*y1)

        #y=mx +c => x=(y-c)/m

        x1=int((y1-intercept)/slope)
        x2=int((y2-intercept)/slope)

        poly_vertices.append((x1, y1))
        poly_vertices.append((x2, y2))

        draw_lines(img, np.array([[[x1,y1,x2,y2]]]))
    
    order = [0,1,3,2]
    poly_vertices = [poly_vertices[i] for i in order]
    cv.fillPoly(img, pts = np.array([poly_vertices],'int32'), color = (0,255,0))
    return cv.addWeighted(image,0.7,img,0.4,0.)

In [ ]:
def transform(img, rho, theta, threshold, min_line_len, max_line_gap):

    lines = cv.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8) 
    line_img = slope_lines(line_img,lines)
    return line_img

In [ ]:
def weighted_img(img, initial_img, α=0.1, β=1., γ=0.):

    lines_edges = cv.addWeighted(initial_img, α, img, β, γ)
    return lines_edges

In [ ]:
def get_vertices(image):

    rows = image.shape[0]
    cols=image.shape[1]

    # the below values have been obtained by hit and trail method (on a test image from the data)
    # top left corner-> is the origin (0,0) in the image
    
    bottom_left  = [cols*0.15, rows]
    top_left     = [cols*0.45, rows*0.6]
    bottom_right = [cols*0.95, rows]
    top_right    = [cols*0.55, rows*0.6] 
    
    ver = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    return ver

In [ ]:
def detect_lane(image):

    #convert BGR to grayscale image
    img= cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    #Gaussian Smooth (to remove the noise)
    smoothed_img = cv.GaussianBlur(img, (5,5), 0) #kernel of size 5

    #Canny Edge 
    canny_img = cv.Canny(smoothed_img,180, 240) #low_threshold = 180, high_threshold = 240

    #Mask image 
    masked_img = bounded_region_mask(img = canny_img, vertices = get_vertices(image))

    #Hough Transform 
    hough_lines = transform(img = masked_img, rho = 1, theta = np.pi/180, threshold = 20, min_line_len = 20, max_line_gap = 180)

    #Draw lines (edges of lanes)
    lane = weighted_img(img = hough_lines, initial_img = image, α=0.8, β=1., γ=0.)
    
    return lane

# Demonstration of Lane Detection

In [ ]:
output = 'new.mp4' #the output will be stored in this

clip = VideoFileClip("/content/test_videos/solidYellowLeft.mp4").subclip(10,30) # a portion of the video has been used

out_clip = clip.fl_image(detect_lane) #function is applied on each frame(image) of the video clip

%time out_clip.write_videofile(output, audio=False) #write to the output file

[MoviePy] >>>> Building video new.mp4
[MoviePy] Writing video new.mp4





  0%|          | 0/501 [00:00<?, ?it/s]


  1%|          | 6/501 [00:00<00:09, 53.02it/s]


  2%|▏         | 12/501 [00:00<00:09, 52.99it/s]


  4%|▎         | 18/501 [00:00<00:09, 53.32it/s]


  5%|▍         | 23/501 [00:00<00:09, 50.80it/s]


  6%|▌         | 29/501 [00:00<00:09, 51.63it/s]


  7%|▋         | 35/501 [00:00<00:09, 51.58it/s]


  8%|▊         | 40/501 [00:00<00:09, 50.72it/s]


  9%|▉         | 45/501 [00:00<00:09, 46.75it/s]


 10%|▉         | 50/501 [00:01<00:14, 32.18it/s]


 11%|█         | 54/501 [00:01<00:17, 25.73it/s]


 12%|█▏        | 58/501 [00:01<00:19, 23.01it/s]


 12%|█▏        | 61/501 [00:01<00:19, 22.66it/s]


 13%|█▎        | 64/501 [00:01<00:22, 19.39it/s]


 13%|█▎        | 67/501 [00:02<00:22, 19.61it/s]


 14%|█▍        | 70/501 [00:02<00:23, 18.66it/s]


 15%|█▍        | 73/501 [00:02<00:22, 18.75it/s]


 15%|█▍        | 75/501 [00:02<00:22, 18.75it/s]


 15%|█▌        | 77/501 [00:02<00:22, 18.58it/s]


 16%|█▌        | 79/501 [00:02<00:24, 

[MoviePy] Done.
[MoviePy] >>>> Video ready: new.mp4 

CPU times: user 8.83 s, sys: 873 ms, total: 9.7 s
Wall time: 25.8 s


In [ ]:
# watch output video

from IPython.display import HTML
from base64 import b64encode
video_path = '/content/new.mp4'

mp4 = open(video_path,'rb').read()
decoded_vid = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src={decoded_vid} type="video/mp4"></video>')